In [2]:
import pandas as pd
df = pd.read_json("anatole_data_test.json")
df.head()

,instruction,input,output
0,Game ID: 793109047 0.10/0.25 (PRR) Ekek - 5 (H...,,checks
1,Game ID: 791882473 0.10/0.25 (PRR) Ekek - 13 (...,,folds
2,Game ID: 793955493 0.25/0.50 (PRR) Gorgons (Ca...,,folds
3,Game ID: 789358065 1/2 (PRR) Longana - 3 (Hold...,,folds
4,Game ID: 786933996 0.25/0.50 (PRR) Gorgons - 7...,,folds


In [4]:
import os
from mistralai.client import MistralClient
os.environ['MISTRAL_API_KEY']='ymjotzDIdJxKj611QnrKt70VEJsHtyGm'
api_key = os.environ.get("MISTRAL_API_KEY")
client = MistralClient(api_key=api_key)

In [47]:
from mistralai.models.chat_completion import ChatMessage
from tqdm import tqdm

retrieved_job = client.jobs.retrieve('2cad6ef8-34b8-4e20-9431-db9ccccb0f1d')
mistral_reponse=[]
for k in tqdm(range(len(df))):
    chat_response = client.chat(
            model="open-mistral-7b", #retrieved_job.fine_tuned_model,
            messages=[ChatMessage(role='system', content='You are a poker player, complete the action.'), ChatMessage(role='user', content=df['instruction'][k])], 
            temperature = 0.,
            max_tokens=2
        )
    mistral_reponse.append(chat_response.choices[0].message.content)

100%|██████████| 3454/3454 [38:09<00:00,  1.51it/s]  


In [29]:
jobs = client.jobs.list()
print(jobs)

data=[Job(id='2cad6ef8-34b8-4e20-9431-db9ccccb0f1d', hyperparameters=TrainingParameters(training_steps=180, learning_rate=0.0001), fine_tuned_model='ft:open-mistral-7b:c6d4dfa8:20240626:2cad6ef8', model='open-mistral-7b', status='SUCCESS', job_type='FT', created_at=1719433586, modified_at=1719434336, training_files=['5cd53dbf-4f32-45e6-9b15-0c69b05b2cc6'], validation_files=['ff6b09ef-e7ba-4c1f-8a25-a35681ef8c93'], object='job', integrations=[])] object='list'


In [48]:
mistral_reponse

['Checks',
 'Folds',
 'Raises',
 'Raises',
 'checks\n',
 'After receiving',
 'Raises',
 'Player Il',
 'After receiving',
 'Player Il',
 'Calls',
 'Raises',
 'checks\n',
 'Player Il',
 'Player Il',
 'Raises',
 'Player Il',
 'Folds',
 'Raises',
 'Raises',
 'Player Il',
 'calls (',
 'In this',
 'In this',
 'Folds',
 'Player Il',
 'Player Il',
 'bets',
 'Player Il',
 'Calls',
 'bets',
 'Calls',
 'Player Il',
 'Player Il',
 'Calls',
 'calls (',
 'In this',
 'Raises',
 'Player Il',
 'Raises',
 'In this',
 'Raises',
 'Raises',
 'Player Il',
 'Raises',
 'Calls',
 'Player Il',
 'Player Il',
 'Player Il',
 'checks.',
 'Raises',
 'Raises',
 'In this',
 'Player Il',
 'Player Il',
 'Player Il',
 'Player Il',
 'Raises',
 'Raises',
 'calls (',
 'Raises',
 'After receiving',
 'Player Il',
 'After receiving',
 'Calls',
 'Raises',
 'Raises',
 'Calls',
 'Player Il',
 'Folds',
 'After observ',
 'Player Il',
 'Player Il',
 'Raises',
 'Raises',
 'Player Il',
 'Folds',
 'Player Il',
 'Player Il',
 'Folds',
 

In [26]:
%run metrics.ipynb
waited_output=df["output"].to_list()

In [49]:
mistral_reponse= [mistral_reponse[k].split()[0] for k in range(len(mistral_reponse))]


In [50]:
print(compare_to_test(waited_output, mistral_reponse))
print('nombre de coup légal:' , legal_actions(mistral_reponse), len(mistral_reponse))

        bets  calls  raises  allin  checks  folds  caps
bets      14      0       0      0       8      0     0
calls      0     16       1      0       0      0     0
raises     0      3       5      0       0      0     0
allin      0      1       0      0       1      0     0
checks    36      0       0      0      44      0     0
folds      7     87     172      0      48      1     0
caps       0      1       0      0       0      0     0
nombre de coup légal: 429 3454


In [23]:
wrong_indicies = find_misclassifications(waited_output, llm_response)
n=wrong_indicies[3]
print(df['instruction'].iloc[n], "\nwaited_output", waited_output[n], "\nllm response:", llm_response[n] )

Game ID: 808774852 0.25/0.50 (PRR) Gorgons (Cap) - 2 (Hold'em)
Seat 1 is the button
Seat 1: nowhere2be (46.20).
Seat 2: imperceptible (9.51).
Seat 3: elec_365 (63.32).
Seat 4: Jui Kuen (50).
Seat 5: IlxxxlI (34).
Player imperceptible has small blind (0.25)
Player elec_365 has big blind (0.50)
Player IlxxxlI received card: [4c]
Player IlxxxlI received card: [4d]
Player Jui Kuen folds
Player IlxxxlI 
waited_output raises (1) 
llm response: folds


#Sauvegarde 

In [51]:
import csv

# Saving
my_list = mistral_reponse
with open('Mistralnonft_response_test.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(my_list)

# Piepline pour la range pre flop